# F1 Race Prediction
#### **Project**: Predicting the results of F1 races based on historical data
#### **Context**: The growing F1 industry gets more competitive every year, with performance being more calculated than ever. Would it be possible to predict the results?

## 1. Introduction/Business Problem

F1 prediction - does engine play a major impact?

In [1]:
import pandas as pd
import numpy as np

circuits_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\circuits.csv')
constructor_results_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\constructor_results.csv')
constructor_standings_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\constructor_standings.csv')
constructors_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\constructors.csv')
driver_standigs_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\driver_standings.csv')
drivers_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\drivers.csv')
lap_times_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\lap_times.csv')
pit_stops_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\pit_stops.csv')
qualifying_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\qualifying.csv')
races_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\races.csv')
results_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\results.csv')
seasons_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\seasons.csv')
status_df = pd.read_csv(r'C:\Users\joaoe\Documentos\JP\JP_GitHub\f1_predict_ML\f1db_csv\status.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\joaoe\\Documentos\\JP\\JP_GitHub\\f1_predict_ML\\f1db_csv\\circuits.csv'

### Merging races and circuits tables


In [ ]:
# Adding a circuit reference to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['circuitRef'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Circuit Reference'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit location to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['location'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Location'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit country to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['country'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Country'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit latitude to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['lat'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Latitude'] = circuits_in_races.replace(circuits_dict)

# Adding a circuit longitude to each corresponding circuit Id
circuits_dict = circuits_df.set_index('circuitId')['lng'].to_dict()
circuits_in_races = races_df.filter(like='circuitId')
races_df['Longitude'] = circuits_in_races.replace(circuits_dict)


# Droping irrelevant columns
races_df = races_df.drop(columns=['raceId','time','circuitId'])

races_df = races_df[['year', 'date', 'round', 'name', 'Circuit Reference', 'Location', 'Country', 'Latitude','Longitude']]
races_df.rename(columns={'round': 'Round','year': 'Year', 'name': 'Name', 'date': 'Date'})

In [ ]:

from selenium import webdriver

weather = races_df.iloc[:,[0,1,2]]

info = []

# read wikipedia tables

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        # find weather in italian with selenium
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

# append column with weather information to dataframe  
  
weather['weather'] = info

# set up a dictionary to convert weather information into keywords

weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

# map new df according to weather dictionary

weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)
   
weather_info = pd.concat([weather, weather_df], axis = 1)
